In [ ]:
import pandas as pd 
import numpy as np

### 절대모멘텀
- 파일 로드 
- 파생변수 'STD-YM' 생성 -> Date 컬럼의 값을 년-월 추출 대입
- 년,월 별 마지막날 month_last_dt 데이터프레임 삽입 
- 전월의 종가의 값을 가지는 파생변수 생성
- 전년도의 종가의 값을 가지는 파생변수 생성
- 전월, 전년도의 종가를 가지고 거래내역 생성
- 수익율 계산

In [ ]:
df = pd.read_csv("../csv/SPY.csv")
df.head(1)

In [ ]:
## 새로운 파생변수 STD-YM 생성
## Date 컬럼을 기준으로 년-월 추출해서 파생변수 대입

## Date 컬럼을 데이터형을 datetime 변경 ( pd.to_datetime() )
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
## STD-YM 컬럼을 만들어서 Date컬럼에서 년(4자리)-월 추출
df['STD-YM'] = df['Date'].dt.strftime("%Y-%m")

In [ ]:
month_last_df = pd.DataFrame()

month_list = df['STD-YM'].unique()

for i in month_list:
    last_df = df[df['STD-YM'] == i].tail(1)
    month_last_df = pd.concat([month_last_df, last_df])
month_last_df.info()

In [ ]:
## 년-월 별 마지막날의 데이터들을 추출 
## 새로운 데이터프레임에 결합
month_last_df = df[~(df.shift(-1)['STD-YM'] == df['STD-YM'])]
month_last_df.info()

In [ ]:
## 'Date' 컬럼을 인덱스 변환
month_last_df.set_index(['Date'], inplace=True)

In [28]:
month_last_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,STD-YM,BF_1M,BF_12M
Date,,,,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,26.706757,1003200,1993-01,0.000000,0.0
1993-02-26,44.43750,44.43750,44.18750,44.40625,26.991682,66200,1993-02,26.706757,0.0
1993-03-31,45.34375,45.46875,45.18750,45.18750,27.596252,111600,1993-03,26.991682,0.0
1993-04-30,44.12500,44.28125,44.03125,44.03125,26.890142,88500,1993-04,27.596252,0.0
1993-05-28,45.40625,45.40625,45.00000,45.21875,27.615356,79100,1993-05,26.890142,0.0
1993-06-30,45.12500,45.21875,45.00000,45.06250,27.714964,437600,1993-06,27.615356,0.0
1993-07-30,45.09375,45.09375,44.78125,44.84375,27.580431,75300,1993-07,27.714964,0.0
1993-08-31,46.40625,46.56250,46.34375,46.56250,28.637531,66500,1993-08,27.580431,0.0
1993-09-30,46.03125,46.12500,45.84375,45.93750,28.429131,99300,1993-09,28.637531,0.0


In [ ]:
## month_last_df 파생변수 2개 생성
## 'BF_1M' 전월의 종가(Adj Close)의 값
## 'BF_12M' 전년도의 종가(Adj Close)의 값
## NaN이 나온다면 0으로 대체
month_last_df["BF_1M"] = month_last_df.shift(1)['Adj Close'].fillna(0)
month_last_df["BF_12M"] = month_last_df.shift(12)['Adj Close'].fillna(0)
month_last_df.iloc[10:15]

In [30]:
month_last_df.loc['1993-01-29':'1993-09-30']

,Open,High,Low,Close,Adj Close,Volume,STD-YM,BF_1M,BF_12M
Date,,,,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,26.706757,1003200,1993-01,0.000000,0.0
1993-02-26,44.43750,44.43750,44.18750,44.40625,26.991682,66200,1993-02,26.706757,0.0
1993-03-31,45.34375,45.46875,45.18750,45.18750,27.596252,111600,1993-03,26.991682,0.0
1993-04-30,44.12500,44.28125,44.03125,44.03125,26.890142,88500,1993-04,27.596252,0.0
1993-05-28,45.40625,45.40625,45.00000,45.21875,27.615356,79100,1993-05,26.890142,0.0
1993-06-30,45.12500,45.21875,45.00000,45.06250,27.714964,437600,1993-06,27.615356,0.0
1993-07-30,45.09375,45.09375,44.78125,44.84375,27.580431,75300,1993-07,27.714964,0.0
1993-08-31,46.40625,46.56250,46.34375,46.56250,28.637531,66500,1993-08,27.580431,0.0
1993-09-30,46.03125,46.12500,45.84375,45.93750,28.429131,99300,1993-09,28.637531,0.0


In [ ]:
df.loc[1]

### loc, iloc
1. loc 
    - index와 column의 값을 비교하여 True인 행과 열을 출력
    - loc[index, columns]
    - loc[1] -> index의 값이 1인 행 출력
    - loc[:, column명]
2. iloc
    - index와 column의 위치를 출력
    - 데이터프레임에 2번째 행 출력 -> iloc[1]
    - 데이터프레림에서 2번째 열 출력 -> iloc[:,1]

In [ ]:
df.set_index(['Date'], inplace=True)

In [34]:
## 거래내역 추가
## 구매 조건 -> 전월 종가 / 전년도 종가 - 1의 값이 0보다 크고 무한대가 아닌 경우
df['trade'] = ""

for i in month_last_df.index:
    signal = ""

    # 절대 모멘텀 계산
    momentum_index = month_last_df.loc[i, "BF_1M"] /\
         month_last_df.loc[i, "BF_12M"] - 1
    # print(momentum_index)
    # 절대 모멘텀 지표에 따라서 True / False 구분
    flag = True if((momentum_index > 0) and (momentum_index != np.inf) 
    and (momentum_index != -np.inf)) else False

    if flag:
        signal = 'buy'
    print("날짜 :", i, '모멘텀 인덱스 :', momentum_index, 
    "flag :", flag, 'signal :', signal)
    df.loc[i, "trade"] = signal
df['trade'].value_counts()
    


C:\Users\moons\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Users\moons\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


날짜 : 1993-01-29 00:00:00 모멘텀 인덱스 : nan flag : False signal : 
날짜 : 1993-02-26 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-03-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-04-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-05-28 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-06-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-07-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-08-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-09-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-10-29 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-11-30 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1993-12-31 00:00:00 모멘텀 인덱스 : inf flag : False signal : 
날짜 : 1994-01-31 00:00:00 모멘텀 인덱스 : 0.08709114326385636 flag : True signal : buy
날짜 : 1994-02-28 00:00:00 모멘텀 인덱스 : 0.1131289632117034 flag : True signal : buy
날짜 : 1994-03-31 00:00:00 모멘텀 인덱스 : 0.05699081889816027 flag : True signal : buy
날짜 : 1994-04-29 0

       6401
buy     247
Name: trade, dtype: int64

In [35]:
rtn = 1.0
df['return'] = 1
buy = 0
sell = 0
for i in df.index:
    ##구매한 날짜를 체크 : 현재 행의 trade = 'buy' 전 행의 trade = ""인 경우
    if df.loc[i, 'trade'] == 'buy' and df.shift(1).loc[i, 'trade'] == "":
        buy = df.loc[i, 'Adj Close']
        print("구매일 :", i, "구매 가격 :", buy)
    elif df.loc[i, 'trade'] == "" and df.shift(1).loc[i, 'trade'] == 'buy':
        sell = df.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1
        df.loc[i, 'return'] = rtn
        print("판매일 : ", i, "판매 가격 :", sell, "수익율 :", rtn)

    if df.loc[i, 'trade'] == "":
        buy = 0 
        sell = 0 


구매일 : 1994-01-31 00:00:00 구매 가격 : 30.045222999999996
판매일 :  1994-02-01 00:00:00 판매 가격 : 29.889435 수익율 : 0.994814882885043
구매일 : 1994-02-28 00:00:00 구매 가격 : 29.168985
판매일 :  1994-03-01 00:00:00 판매 가격 : 29.052134000000002 수익율 : 0.9959939984198971
구매일 : 1994-03-31 00:00:00 구매 가격 : 27.946764
판매일 :  1994-04-04 00:00:00 판매 가격 : 27.5159 수익율 : 0.9845826872835794
구매일 : 1994-04-29 00:00:00 구매 가격 : 28.260098
판매일 :  1994-05-02 00:00:00 판매 가격 : 28.436377 수익율 : 1.0062377349151443
구매일 : 1994-05-31 00:00:00 구매 가격 : 28.710556
판매일 :  1994-06-01 00:00:00 판매 가격 : 28.837846999999996 수익율 : 1.00443359578268
구매일 : 1994-06-30 00:00:00 구매 가격 : 28.052668
판매일 :  1994-07-01 00:00:00 판매 가격 : 28.111818 수익율 : 1.0021085338478322
구매일 : 1994-07-29 00:00:00 구매 가격 : 28.959521999999996
판매일 :  1994-08-01 00:00:00 판매 가격 : 29.097497999999998 수익율 : 1.0047644432805212
구매일 : 1994-08-31 00:00:00 구매 가격 : 30.063478000000003
판매일 :  1994-09-01 00:00:00 판매 가격 : 29.964916 수익율 : 0.9967215370091244
구매일 : 1994-09-30 00:00:00 구매 가격 : 29.30

In [36]:
acc_rtn = 1 

for i in df.index:
    rtn = df.loc[i, 'return']
    acc_rtn *= rtn
    df.loc[i, 'acc_rtn'] = acc_rtn

print('누적 수익율 :', acc_rtn)

누적 수익율 : 2.0162118510060383


### 절대 모멘텀 함수로 구현
1. 1번 함수
    - 매개변수 2개 ( 데이터프레임, 기준이 되는 컬럼 )
    - 데이터프레임에 결측치와 이상치 제거
    - Date, 기준이 되는 컬럼 두 컬럼만 두고 나머지 컬럼은 삭제
    - Date 컬럼의 데이터를 시계열데이터 변경
    - 'STD-YM' 파생변수를 생성을 하여 년-월 대입
    - 인덱스를 Date로 변경
    - 수정이 된 데이터프레임을 리턴
    - 1번 함수 테스트하실때는 csv 파일 새로 불러와서 함수호출
2. 2번 함수
    - 매개변수(df) 1개( 데이터프레임 )
    - 새로운 데이터프레임 생성 
    - df의 데이터에서 년-월별 마지막 데이터들을 새로운 데이터프레임 삽입
    - 새로운 데이터프레임에 파생변수 2개 생성
    - 'BF_1M' : 전 월의 종가, 결측치는 0으로 대체
    - 'BF_12M' : 전 년의 종가, 결측치는 0으로 대체
    - 새로운 데이터프레임을 리턴


In [37]:
## 1번 함수 생성
def first(data, col):
    ## 결측치와 이상치를 제거
    result = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)]
    result = result.loc[:, ['Date', col]] ## result[['Date', col]]
    result['Date'] = pd.to_datetime(result['Date'])
    result['STD-YM'] = result['Date'].dt.strftime("%Y-%m")
    result.set_index('Date', inplace=True)
    return result

In [40]:
test_df = pd.read_csv("../csv/AAPL.csv")

test_df = first(test_df, 'Close')

In [39]:
## 2번 함수
def second(data):
    result = pd.DataFrame()
    col = data.columns[0]
    data = data[~(data.shift(-1)['STD-YM'] == data['STD-YM'])]
    result = pd.concat([result, data])
    result['BF_1M'] = result.shift(1)[col].fillna(0)
    result['BF_12M'] = result.shift(12)[col].fillna(0)
    return result
    

In [42]:
test_df_2 = second(test_df)